# Combining mask files

Written by K. Garner, 2020
This code will take the relevant mask files from the Keuken atlas, and combine them to make a single bilateral mask file for each ROI

In [1]:
import pandas as pd
import os, re, json, glob
# import nipype.algorithms.confounds as cfg - not using because of bug https://neurostars.org/t/nipype-isdefined-error-when-using-combine-mask-files/6888
import nibabel as nib
import numpy as np

In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
Keuken = ''.join([Basedir, 'derivatives/roi-masks/Keuken/lateralised/*.nii.gz'])
KeukenAll = glob.glob(Keuken)
KeukenAll

['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/lateralised/RSN.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/lateralised/RSTN.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/lateralised/LSTN.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/lateralised/LGPi.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/lateralised/LSN.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/lateralised/LCN.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/lateralised/RCN.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/lateralised/LGPe.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/lateralised/RP.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/lateralised/RRN.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/lateralised/LRN.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-mas

Now I will make some 'keys' of reg expressions that will grab the correct 2 files I need, for each application of the 'combine_mask_files' function

In [3]:
keys = ['*STN.', '*CN.', '*VS.', '*GPe.', '*GPi.', '*P.']
labels = [ '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/STN.nii.gz', 
           '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/CN.nii.gz', 
           '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/VS.nii.gz', 
           '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/GPe.nii.gz', 
           '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/GPi.nii.gz',
           '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Keuken/Put.nii.gz']
mask_pattern = 'derivatives/roi-masks/Keuken/lateralised/'
mask_pattern

'derivatives/roi-masks/Keuken/lateralised/'

And define a function for combining mask files:

For some reason, the combine_mask_files function fails on its first logical definition, so pasting code across from function to use manually...https://github.com/nipy/nipype/blob/1.4.2/nipype/algorithms/confounds.py

In [4]:
def combine_mask_files_kg(key, mask_pattern, fname, Basedir):
    # key = a reg exp to find specific ROIs
    # mask pattern = a template to find file names using glob
    # fname = save filename
    # combines using the union method
    fs = ''.join([Basedir, mask_pattern, key, 'nii.gz'])
    mask_files = glob.glob(fs)
    mask = None
    for filename in mask_files:
        img = nib.load(filename)
        img_as_mask = np.asanyarray(img.dataobj).astype("int32") > 0
        if mask is None:
            mask = img_as_mask
    np.logical_or(mask, img_as_mask, mask)
    img = nib.Nifti1Image(mask, img.affine, header=img.header)
    nib.save(img, fname)

Now I apply the function over the key and label lists


In [ ]:
for key, label in zip(keys, labels):
    combine_mask_files_kg(key=key, mask_pattern=mask_pattern, fname=label, Basedir=Basedir)



et viola! :)

## And now the same for the cortical mask files

In [5]:
Cort = ''.join([Basedir, 'derivatives/roi-masks/Cort/lateralized/*.nii.gz'])
Cort = glob.glob(Cort)
Cort

['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Cort/lateralized/LIPSsphere_bin.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Cort/lateralized/RIPSsphere_bin.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Cort/lateralized/RFEFsphere_bin.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Cort/lateralized/LOCsphere_bin.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Cort/lateralized/LFEFsphere_bin.nii.gz']

In [20]:
keys = ['*FEF*', '*IPS*']
labels = [ '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Cort/FEF.nii.gz', 
           '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Cort/IPS.nii.gz']
mask_pattern = 'derivatives/roi-masks/Cort/lateralized/'
mask_pattern

'derivatives/roi-masks/Cort/lateralized/'

In [21]:
for key, label in zip(keys, labels):
    combine_mask_files_kg(key=key, mask_pattern=mask_pattern, fname=label, Basedir=Basedir)